## Importando os dados

    O objetivo deste projeto é desenvolver um modelo que possa analisar o comportamento do cliente e recomendar um dos planos mais recentes da Megaline: Smart ou Ultra.
     Para tanto, vamos realizar uma tarefa de classificação, e desenvolver um modelo que escolherá o plano certo, e  com a maior acurácia possível. 
    Para iniciar temos acesso a dados de comportamento dos assinantes que já mudaram para os novos planos. Para esta tarefa de classificação precisamos desenvolver um modelo que escolherá o plano certo. e com a maior acurácia possível, definido em 0,75. 

In [1]:
import pandas as pd 
import scipy.stats  as spt 
import seaborn as sns
import matplotlib as mplt
import matplotlib.pyplot as plt
import seaborn as sns
import  plotly.express as px
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error


In [2]:
df= pd.read_csv('/datasets/users_behavior_upd.csv')

In [3]:
df.head(5)

,calls,minutes,messages,mb_used,is_ultimate
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


Este conjunto de dados traz informações sobre o comportamento dos usuários e a utilizaão dos serviços, bem como uma coluna (is_ultimate) que traz o resultado de uma classificação em que 1- significa que o usuário ja tem o plano Ultimate, e 0- que ele tem outro plano (surf).

### Dividindo o dataset em train, test e valid

     A variável que queremos estudar como objetivo é 'is_ultimate', que será definida como o target. 
 
     As demais colunas serão nossas features, que são compostas todas por dados numéricos. 

In [4]:
features= df.drop(df['is_ultimate']).values 

In [5]:
target= df['is_ultimate'].values

   * Vamos dividir os dados em conjuntos de treinamento, validação e teste: 
      * A primeira divisão será o que vai armazernar os dados de test para uma verificação da qualidade do modelo. 
      * O restante será utilizado para formar uma nova divisão que resultará nos conjuntos de treino e validação.
      * O valor escolhido para o tamanho do conjunto de teste é de 25% do conjunto original, o que é suficiente para uma validação da qualidade.


In [6]:
df_train_valid, df_test = train_test_split(df, test_size=0.25, random_state=12345)

In [7]:
df_train, df_valid = train_test_split(df_train_valid, test_size=0.25, random_state=12345)


In [8]:
features_train = df_train.drop(['is_ultimate'], axis=1)
target_train = df_train['is_ultimate']
features_valid = df_valid.drop(['is_ultimate'], axis=1)
target_valid = df_valid['is_ultimate']
features_test = df_test.drop(['is_ultimate'], axis=1)
target_test = df_test['is_ultimate']

### Iniciando os modelos de regressão 

 * Vamos testar como os dados deste dataset reagem à alguns dos diferentes modelos existentes. 

####        Regressão tipo DecisionTree 

In [9]:
model_tree = DecisionTreeClassifier(random_state=12345)


In [10]:
model_tree.fit(features_train, target_train)

DecisionTreeClassifier(random_state=12345)

In [11]:

best_model = None
best_result = 0
for depth in range(1, 6):
	model_tree = DecisionTreeClassifier(random_state=12345, max_depth= depth ) 
	model_tree.fit(features_train, target_train) 
	predictions = model_tree.predict(features_valid)
	result = accuracy_score(target_valid,predictions) 
	if result > best_result:
		best_model = features_train
		best_result = result
predict_train = model_tree.predict(features_train)

print("max_depth =", depth, ": ", end='')
print("Train:", accuracy_score(target_train,predict_train))
print("Valid:", accuracy_score(target_valid,predictions))
print("Acurácia do melhor modelo:", best_result)


max_depth = 5 : Train: 0.8201438848920863
Valid: 0.7877280265339967
Acurácia do melhor modelo: 0.7943615257048093


   * A acurácia dos 2 conjuntos tem uma diferença pequena. O resultado do melhor modelo é satisfatório para nosso limite de corte aceitavel de >0,75.
      - Como resultado,o melhor valor para o parâmetro max_depth foi de =5. 
 Resta compararmos este resultado com sos demais modelos para, ai então escolher o melhor modelo. 

####        Regressão tipo  RandomForest 

In [12]:
model_rfc = RandomForestClassifier(random_state=54321, n_estimators=9)

In [13]:
model_rfc.fit(features_train, target_train)

RandomForestClassifier(n_estimators=9, random_state=54321)

In [14]:
valid_predictions_rfc = model_rfc.predict(features_valid)
valid_accur_score_rfc= accuracy_score(target_valid, valid_predictions_rfc)

test_predictions_rfc= model_rfc.predict(features_test) 
test_predict_accur_rfc= accuracy_score(target_test, test_predictions_rfc)

In [15]:
print('Acurácia' )
print('Conjunto de teste:',  test_predict_accur_rfc)
print('Conjunto de valid:',  valid_accur_score_rfc)


Acurácia
Conjunto de teste: 0.7823383084577115
Conjunto de valid: 0.7927031509121062


In [16]:
best_score = 0
best_est = 0  
for est in range(1, 11): 
    model_rfc = RandomForestClassifier(random_state=54321, n_estimators=est) 
    model_rfc.fit(features_train, target_train ) 
    predict_rfc = model_rfc.predict(features_valid)
    score = accuracy_score(target_valid, predict_rfc)
    if score > best_score:
        best_score = score 
        best_est = est 

predict_rfc_train = model_rfc.predict(features_train)
        
print("Train:", accuracy_score(target_train,predict_rfc_train))
print("Valid:", accuracy_score(target_valid,predict_rfc))
print("Acurácia do melhor modelo no conjunto de valid (n_estimators = {}): {}".format(best_est, best_score))
        

Train: 0.9800774764803541
Valid: 0.7976782752902156
Acurácia do melhor modelo no conjunto de valid (n_estimators = 8): 0.7993366500829188


    Observações sobre o Modelo:
- As acurácias tiveram uma diferença grande entre de valid e train, o que mostra que o modelo não é tão preciso.
- Obtivemos também um melhor valor para o parâmetro n_estimators, de =8. 

####        regressão tipo Logistic regression

In [17]:
model_lr = LogisticRegression(random_state=12345) 
model_lr.fit(features_train, target_train) 

LogisticRegression(random_state=12345)

In [18]:
valid_pred_lr = model_lr.predict(features_valid)
valid_acc_lr= accuracy_score(target_valid, valid_pred_lr)

In [20]:
('Acurácia' )
print('Conjunto de validação:',  valid_acc_lr)


Conjunto de validação: 0.7412935323383084


In [21]:
result = mean_squared_error(target_valid, valid_pred_lr)**0.5 
print("O REQM do modelo de regressão linear no conjunto de validação:", result)

O REQM do modelo de regressão linear no conjunto de validação: 0.5086319569803804


    Observações sobre o Modelo:

 * Esse modelo foi o que teve a menor acurácia, e como ficou abaixo a nossa linha de corte (0,75), vamos descartá-lo.  

### TREINANDO MODELO COM MELHOR ACURÁCIA

       Os 2 Modelos DecisionTreeClassifier E RandomForest tiveram o resultado do teste de accurácia com o valores muito próximos, de= 0.788 e 0.79 respectivamente. 
       Mas o modelo DecisinTree foi o que teve o resultado mais próximo do valor de acurácia do conjunto de train, o que demonstra que este modelo tem maior precisão.  
  Sendo assim vamos rodar o Modelo final utilizando a regressão tipo DecisionTreeClassifier, com o hiperparâmetre max_depth = 5 e obter a acurácia do conjunto de teste como forma de verificação da qualidade do modelo, utilizando um dado não enviesado (que não foi treinado) .
         Ao se rodar o modelo, foi utilizado um loop for para obter o melhor resultado do hiperparâmetro que teve como resultado: max_depth = 5. 


In [22]:
model_dtc_fin = DecisionTreeClassifier(random_state=12345, max_depth= 5 ) 

In [23]:
model_dtc_fin.fit(features_train, target_train)

DecisionTreeClassifier(max_depth=5, random_state=12345)

In [24]:
pred_test = model_dtc_fin.predict(features_test)
accurc_test = accuracy_score(target_test,pred_test)

In [25]:
print('Acurácia' )
print('Conjunto final de validação:',  accurc_test)

Acurácia
Conjunto final de validação: 0.7873134328358209


Como já esperado, a acurácia do modelo de teste (0.78) é muito próxima ao modelo de validação (0.78), o que indica uma qualidade satisfatória pára o modelo. 